In [ ]:
"# Databricks notebook source\n# COMMAND ----------\nimport logging\nfrom pyspark.sql import functions as F\nfrom pyspark.sql.types import DoubleType\n\n# Configure logging\nlogging.basicConfig(level=logging.INFO)\nlogger = logging.getLogger(__name__)\n\n# COMMAND ----------\ntry:\n    # Step 1: Data Loading\n    logger.info(\"Loading data from Unity Catalog tables.\")\n    policy_df = spark.table(\"genai_demo.guardian.policy\")\n    claims_df = spark.table(\"genai_demo.guardian.claims\")\n    demographics_df = spark.table(\"genai_demo.guardian.demographics\")\n    scores_df = spark.table(\"genai_demo.guardian.scores\")\n    aiml_insights_df = spark.table(\"genai_demo.guardian.aiml_insights\")\n\n    # COMMAND ----------\n    # Step 2: Data Selection and Filtering\n    logger.info(\"Selecting necessary fields from each dataset.\")\n    demographics_selected = demographics_df.select(\n        \"Customer_ID\", \"Customer_Name\", \"Email\", \"Phone_Number\", \"Address\", \"City\", \"State\", \"Postal_Code\", \n        \"Date_of_Birth\", \"Gender\", \"Marital_Status\", \"Occupation\", \"Income_Level\", \"Customer_Segment\"\n    )\n\n    claims_selected = claims_df.select(\n        \"Claim_ID\", \"Policy_ID\", \"Claim_Date\", \"Claim_Type\", \"Claim_Status\", \"Claim_Amount\", \"Claim_Payout\"\n    )\n\n    policy_selected = policy_df.select(\n        \"policy_id\", \"customer_id\", \"policy_type\", \"policy_status\", \"policy_start_date\", \"policy_end_date\", \n        \"policy_term\", \"policy_premium\", \"total_premium_paid\", \"renewal_status\", \"policy_addons\"\n    )\n\n    # COMMAND ----------\n    # Step 3: Data Integration\n    logger.info(\"Joining datasets on common identifiers.\")\n    customer_policy_join = demographics_selected.join(policy_selected, demographics_selected.Customer_ID == policy_selected.customer_id, \"inner\")\n    customer_policy_claims_join = customer_policy_join.join(claims_selected, customer_policy_join.policy_id == claims_selected.Policy_ID, \"inner\").drop(claims_selected.Policy_ID)\n\n    # COMMAND ----------\n    # Step 4: Data Aggregation\n    logger.info(\"Aggregating data to calculate metrics.\")\n    aggregated_data = customer_policy_claims_join.groupBy(\"Customer_ID\").agg(\n        F.count(\"Claim_ID\").alias(\"Total_Claims\"),\n        F.count(\"policy_id\").alias(\"Policy_Count\"),\n        F.max(\"Claim_Date\").alias(\"Recent_Claim_Date\"),\n        F.avg(\"Claim_Amount\").alias(\"Average_Claim_Amount\")\n    )\n\n    # COMMAND ----------\n    # Step 5: Custom Calculations\n    logger.info(\"Implementing custom calculations for additional metrics.\")\n    final_data = aggregated_data.join(customer_policy_claims_join, \"Customer_ID\", \"inner\").withColumn(\n        \"Age\", F.datediff(F.current_date(), F.col(\"Date_of_Birth\")) / 365\n    ).withColumn(\n        \"Claim_To_Premium_Ratio\", F.when(F.col(\"total_premium_paid\") != 0, F.col(\"Claim_Amount\").cast(DoubleType()) / F.col(\"total_premium_paid\").cast(DoubleType())).otherwise(0)\n    ).withColumn(\n        \"Claims_Per_Policy\", F.when(F.col(\"Policy_Count\") != 0, F.col(\"Total_Claims\") / F.col(\"Policy_Count\")).otherwise(0)\n    ).withColumn(\n        \"Retention_Rate\", F.lit(0.85)\n    ).withColumn(\n        \"Cross_Sell_Opportunities\", F.lit(\"Multi-Policy Discount, Home Coverage Add-on\")\n    ).withColumn(\n        \"Upsell_Potential\", F.lit(\"Premium Vehicle Coverage\")\n    )\n\n    # COMMAND ----------\n    # Step 6: Final Data Assembly\n    logger.info(\"Combining all processed data into a single dataset.\")\n    final_customer_360 = final_data.join(scores_df, \"Customer_ID\", \"inner\").join(aiml_insights_df, \"Customer_ID\", \"inner\")\n\n    # COMMAND ----------\n    # Step 7: Output Data\n    logger.info(\"Writing the final dataset to a Delta table.\")\n    final_customer_360.write.format(\"delta\").mode(\"overwrite\").saveAsTable(\"genai_demo.guardian.customer_360\")\n\nexcept Exception as e:\n    logger.error(\"An error occurred during the ETL process: %s\", e)\n    raise\n"
